In [17]:
import time

import requests
import re
import json
import os
import string
import hashlib

import bs4
from bs4 import BeautifulSoup
import tqdm 

# Retrieve All Embassy Website URLs

We first retrieve all the urls to each embassy's website.

In [2]:
us_embassy_main_url = 'https://www.usembassy.gov/post-sitemap.xml'
us_embassy_main_request = requests.get(us_embassy_main_url)
us_embassy_main_request

<Response [200]>

In [3]:
us_embassy_main_html = us_embassy_main_request.content
us_embassy_main_soup = BeautifulSoup(us_embassy_main_html, 'lxml')

In [4]:
embassy_url_list = [loc.string for loc in us_embassy_main_soup.find_all('loc')]
embassy_url_list[:5]

['https://www.usembassy.gov/south-africa/',
 'https://www.usembassy.gov/indonesia/',
 'https://www.usembassy.gov/malaysia/',
 'https://www.usembassy.gov/laos/',
 'https://www.usembassy.gov/marshall-islands/']

In [95]:
def embassy_url_prefix(x):
    a = re.match("(http|https)://(?!www).*\.(usmission|usembassy|usconsulate).*", x['href'])
    return a.group() if a else None

In [52]:
embassy_url_map = {}
for embassy in embassy_url_list:
    country_name = embassy.split('/')[-2]
    print(embassy.split('/')[-2])
    embassy_request = requests.get(embassy)
    embassy_html = embassy_request.content
    embassy_soup = BeautifulSoup(embassy_html, 'lxml')
    # cities = embassy_soup.find(class_='cityname1')
    # country_website = '/'.join(cities.a['href'].split('/')[:3])
    country_website_list = list(
        filter(
            None,
            list(
                map(
                    embassy_url_prefix,
                    embassy_soup.find_all('a')
                )
            )
        )
    )
    if len(country_website_list)>0:
        country_website = country_website_list[0]
        country_website = '/'.join(country_website.split('/')[:3])
        print(country_website)
        embassy_url_map[country_name] = country_website
    time.sleep(2)

south-africa
https://za.usembassy.gov
indonesia
https://id.usembassy.gov
malaysia
https://my.usembassy.gov
laos
https://la.usembassy.gov
marshall-islands
https://mh.usembassy.gov
samoa
https://ws.usembassy.gov
thailand
https://th.usembassy.gov
vietnam
https://vn.usembassy.gov
austria
https://at.usembassy.gov
belarus
https://by.usembassy.gov
czech-republic
https://cz.usembassy.gov
denmark
https://dk.usembassy.gov
france
https://fr.usembassy.gov
ireland
https://ie.usembassy.gov
italy
https://it.usembassy.gov
latvia
https://lv.usembassy.gov
malta
https://mt.usembassy.gov
poland
https://pl.usembassy.gov
romania
https://ro.usembassy.gov
sweden
https://se.usembassy.gov
united-kingdom
https://uk.usembassy.gov
jordan
https://jo.usembassy.gov
saudi-arabia
https://sa.usembassy.gov
bangladesh
https://bd.usembassy.gov
kazakhstan
https://kz.usembassy.gov
pakistan
https://pk.usembassy.gov
sri-lanka
https://lk.usembassy.gov
argentina
https://ar.usembassy.gov
chile
https://cl.usembassy.gov
ecuador
htt

## Missing URLs

The `embassy_url_prefix` function is not exhaustive. There are a few remaining embassies left to categorize, which we do manually.

In [86]:
def embassy_country(embassy_url):
    country_name = embassy_url.split('/')[-2]
    return country_name
missing_embassies = []
# set(map(d, embassy_url_list)) - set(embassy_url_map.keys())
for i, country in list(enumerate(embassy_url_list)):
    if embassy_country(country) not in set(embassy_url_map.keys()):
        missing_embassies.append(country)
missing_embassies

['https://www.usembassy.gov/hong-kong/',
 'https://www.usembassy.gov/guatemala/',
 'https://www.usembassy.gov/bermuda/',
 'https://www.usembassy.gov/curacao/',
 'https://www.usembassy.gov/usau/',
 'https://www.usembassy.gov/taiwan-2/']

In [97]:
for url in missing_embassies:
    country_name = d(url)
    url_request = requests.get(url)
    url_html = url_request.content
    url_soup = BeautifulSoup(url_html, 'lxml')
    url_links = list(
        filter(
            None,
            list(
                map(
                    embassy_url_prefix,
                    url_soup.find_all('a')
                )
            )
        )
    )
    print(country_name)
    if len(url_links)>0:
        country_website = url_links[0]
        country_website = '/'.join(country_website.split('/')[:3])
        print(country_website)
        embassy_url_map[country_name] = country_website
    time.sleep(2)

hong-kong
https://hk.usconsulate.gov
guatemala
http://gt.usembassy.gov
bermuda
https://bm.usconsulate.gov
curacao
https://cw.usconsulate.gov
usau
taiwan-2


In [98]:
set(map(d, embassy_url_list)) - set(embassy_url_map.keys())

{'taiwan-2', 'usau'}

In [99]:
embassy_url_map['usau'] = 'https://www.usau.usmission.gov/'
embassy_url_map['taiwan-2'] = 'https://www.ait.org.tw'

## Save/Load your Work

In [101]:
with open("embassy_url_map.json", 'w') as f:
    json.dump(embassy_url_map, f, indent=6)

In [7]:
with open(os.path.join(os.getcwd(), 'data', "embassy_url_map.json"), 'r') as f:
    embassy_url_map = json.load(f)
list(embassy_url_map.items())[:5]

[('south-africa', 'https://za.usembassy.gov'),
 ('indonesia', 'https://id.usembassy.gov'),
 ('malaysia', 'https://my.usembassy.gov'),
 ('laos', 'https://la.usembassy.gov'),
 ('marshall-islands', 'https://mh.usembassy.gov')]

# Retrieve Posts from a Specific Country

## First, we need to get all the existing posts of an embassy's website

In [8]:
target_country = 'south-africa'
sitemap_url = f"{embassy_url_map[target_country]}/post-sitemap.xml"
sitemap_request = requests.get(sitemap_url)
sitemap_html = sitemap_request.content
sitemap_soup = BeautifulSoup(sitemap_html, 'lxml')
embassy_posts = [loc.string for loc in sitemap_soup.find_all('loc')]
embassy_posts

['https://za.usembassy.gov/pepfar-nike-foundation-and-gates-foundation-partnership-restores-hope-for-an-aids-free-future-for-girls/',
 'https://za.usembassy.gov/long-fight-with-tb-the-story-of-thabo-pelesane/',
 'https://za.usembassy.gov/the-power-of-partnership-extraordinary-progress-lessons-learned-and-great-hope-for-the-future-in-south-africa/',
 'https://za.usembassy.gov/pepfar-stories-of-survival-in-rural-mpumalanga-province/',
 'https://za.usembassy.gov/siyayinqoba-beat-it-conquers-fear-of-hiv-testing/',
 'https://za.usembassy.gov/desmond-tutu-blesses-panel-from-aids-quilt-returning-to-washington/',
 'https://za.usembassy.gov/thibela-tb-has-miners-covered/',
 'https://za.usembassy.gov/icap-leaves-proud-legacy-in-eastern-cape/',
 'https://za.usembassy.gov/pepfar-partner-egpafs-successful-project-transitions-to-south-african-organizations/',
 'https://za.usembassy.gov/sm_090415/',
 'https://za.usembassy.gov/sm_200815/',
 'https://za.usembassy.gov/updated-worldwide-caution/',
 'http

## Reading all posts of an embassy's website

In [19]:
data_path = os.path.join(os.getcwd(), 'data', target_country)
if not os.path.exists(data_path):
    os.makedirs(data_path)
for i, embassy_link in enumerate(tqdm.tqdm(embassy_posts)):
    r_post = requests.get(embassy_link)
    html_post = r_post.content
    soup_post = BeautifulSoup(html_post, 'lxml')
    
    post_content = ''
    
    post_title = soup_post.find(class_='mo-breadcrumbs').find('h1').string.strip()
    post_title = post_title.translate(str.maketrans('', '', string.punctuation))
    post_title_hash = hashlib.sha1(post_title.encode('utf-8')).hexdigest()
    
    f = open(
        os.path.join(
            data_path,
            post_title_hash,
        ),
        'w'
    )
    
    f.write(post_title + '\n') 
    for sibling in soup_post.find(class_='main').article.find(class_='entry-content').div.next_siblings:
        if type(sibling) is bs4.element.Tag:
            if sibling.name == 'p':
                text = f"{sibling.text if sibling.text else ''} "
                if sibling.attrs:
                    if 'class' in sibling.attrs:
                        if 'byline' not in sibling.attrs['class']:
                            f.write(text)
                else:
                    f.write(text)
    # print(post_content+"\n")
    f.close()
    time.sleep(5)


 16%|█▌        | 150/950 [20:33<2:03:11,  9.24s/it]


 31%|███▏      | 298/950 [39:12<1:44:23,  9.61s/it]


 47%|████▋     | 446/950 [1:01:38<1:11:12,  8.48s/it]


 62%|██████▏   | 591/950 [1:22:13<53:37,  8.96s/it]


 78%|███████▊  | 739/950 [1:44:01<31:04,  8.83s/it]


 93%|█████████▎| 887/950 [2:05:54<09:32,  9.09s/it]


100%|██████████| 950/950 [2:15:04<00:00,  8.53s/it]


In [117]:
print(soup_post.find(class_='mo-breadcrumbs').find('h1').string.strip())
for sibling in soup_post.find(class_='main').article.find(class_='entry-content').div.next_siblings:
    if type(sibling) is bs4.element.Tag:
        if sibling.name == 'p':
            if sibling.attrs:
                if 'class' in sibling.attrs:
                    if 'byline' not in sibling.attrs['class']:
                        print(sibling.text if sibling.text else '')
            else:
                print(sibling.text if sibling.text else '')

Siyayinqoba: Beat It! Conquers Fear of HIV Testing
Siyayinqoba Beat It!, a South African magazine show produced by the Community Health Media Trust (CMT) discusses hard-hitting topics about people living with HIV.
Not content with using a mass media to reach out on a national level, CMT goes one step further: with funding from PEPFAR through the Centers for Disease Control and Prevention the team has rolled out a comprehensive social mobilization campaign using the catchy Siyayinqoba brand at all levels of media, starting at national and reaching out to local communities through thought-provoking radio talk shows, news articles and even taxis branded with HIV prevention messaging.
CMT’s new project team, based in rural Mthatha in the Eastern Cape, works with local NGOs and community platforms to create a communication cascade that ensures the flow of HIV prevention and health information to all corners of the communities in the predominantly isiXhosa province.
CMT works closely with TB

In [172]:
for s in soup_post.find("main").article.find(class_='entry-content').div.next_siblings:
    print(s.name, s.string, type(s))
    if s.name == 'p':
        if 'class' in s.attrs:
            print('Attributes:', s.attrs['class'])
            if 'byline' not in s.attrs['class']:
                print('Hello!', s.text)
        else:
            print('Hello!', s.text)
        print()

figure None <class 'bs4.element.Tag'>
None 
 <class 'bs4.element.NavigableString'>
p None <class 'bs4.element.Tag'>
Hello! Della Mae Live in Majuro! Embassy Majuro welcomed Nashville-based bluegrass band Della Mae with traditional Marshallese songs and dances performed by Youth to Youth in Health.  Band members spent the week holding music workshops with local youth around town.  Their visit will culminate Saturday night with a free all ages concert in Delap Park at 7 pm.  Della Mae’s visit to Majuro is part of the American Music Abroad program.

None 
 <class 'bs4.element.NavigableString'>
None  AddThis Advanced Settings above via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Advanced Settings below via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Advanced Settings generic via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Share Buttons above via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Share Butt

In [171]:
'byline' in soup_post.find("main").article.find(class_='entry-content').find(class_='byline').attrs['class']

True

In [162]:
soup_post.find("main").article.find(class_='entry-content').attrs

{'class': ['entry-content']}